### 构建算法模型所需数据表（SQL自助分析）
SourceTable: 对原数据表进行清洗，为后续建表的基础。数据清洗的操作有：

    1. 删除卡号为0的数据
    2. 删除实际金额为负数，但商品种类名称为空值的数据
    3. 删除卡类型为不记名卡的数据
    
AggTable: 涵盖所有需聚合计算的指标，包括但不限于：
    实际金额、首次交易日期、是否睡眠卡等
    
RowNumber: 包含行数信息

ParameterTable: 包含聚合计算的指标及业务规则指标，为算法模型的基础表

In [ ]:
-- 构建模型分析所需数据表

-- 原表数据清洗
WITH SourceTable AS
    (
    SELECT 
    Sources.*,
    c_os_card_tbl_trade_m_new_orc_new_g020.occuretime
    FROM
        (
        SELECT *  
        FROM app_gsms_kjyxxb_2020_all_test 
        WHERE 
            NOT(`类型名称` LIKE '%不记名卡%') 
            AND `卡号` <> 0 
            AND NOT(`商品类型` IS NULL AND `实际金额` < 0)
            ) Sources
    LEFT JOIN c_os_card_tbl_trade_m_new_orc_new_g020 
    ON Sources.`交易流水号` = c_os_card_tbl_trade_m_new_orc_new_g020.id
    ),

    
-- 聚合函数、最早最近交易日期、睡眠卡、加油卡基本信息   
AggTable AS
    (
    SELECT agg.*,
    d03.`类型名称`,
    e03.`交易类型描述`,
    f03.`商品类型`,
    g03.`商品种类名称`,
    h03.`单位名称`,
    dates.`首次交易日期`,
    dates.`最近交易日期`,
    c04.`是否为睡眠卡`
    FROM
        (
        SELECT 
            `帐号`,
            `卡号`, 
            AVG(`油量升数`)      `平均油量升数`,
            ABS(AVG(`实际金额`)) `平均消费金额`,
            ABS(AVG(`折扣金额`)) `平均折扣金额`,
            AVG(`余额`)          `平均余额`
        FROM SourceTable
        WHERE `实际金额` < 0
        GROUP BY 
            `帐号`,
            `卡号` 
        HAVING AVG(`油量升数`) <> 0 
        ) agg 
    JOIN 
        (
        SELECT 
        a02.`卡号`,
        `首次交易日期`,
        `最近交易日期`
        FROM
            (
            SELECT 
            `卡号`, 
            `交易日期` `首次交易日期` 
            FROM
                (
                 SELECT 
                 `卡号`, 
                 `交易日期`, 
                 ROW_NUMBER() OVER(PARTITION BY `卡号` ORDER BY `交易日期` ASC) row_number01 
                 FROM SourceTable 
                 WHERE `实际金额` < 0
                ) a01 
            WHERE row_number01 = 1
            ) a02 
        JOIN
            (
            SELECT `卡号`,
            `交易日期` `最近交易日期`
            FROM
                (
                SELECT `卡号`, 
                `交易日期`, 
                ROW_NUMBER() OVER(PARTITION BY `卡号` ORDER BY `交易日期` DESC) row_number02 
                FROM SourceTable 
                WHERE `实际金额` < 0
                ) b01
            WHERE row_number02 = 1
            ) b02 
            ON a02.`卡号` = b02.`卡号` 
        ) dates
    ON agg.`卡号` = dates.`卡号`
    JOIN
        (
        SELECT 
            `卡号`, 
            SUM(`是否睡眠`) `是否为睡眠卡` 
        FROM 
            (
            SELECT 
                `卡号`, 
                CASE WHEN DATEDIFF(`交易日期`,`上次交易日期`) >= 180 THEN 1 ELSE 0 END `是否睡眠` 
                FROM 
                (
                SELECT 
                    `卡号`, 
                    `交易日期`, 
                    LAG(`交易日期`, 1, 0) OVER(PARTITION BY `卡号` ORDER BY row_sleeping ASC) `上次交易日期`, 
                    row_sleeping 
                FROM 
                    (
                    SELECT 
                        `卡号`, 
                        `交易日期`, 
                        ROW_NUMBER() OVER(PARTITION BY `卡号` ORDER BY `交易日期` ASC) row_sleeping 
                    FROM SourceTable
                    ) c01 
                ) c02 
            WHERE row_sleeping <> 1
            ) c03 
        GROUP BY `卡号`
        ) c04
    ON agg.`卡号` = c04.`卡号`
    JOIN
        (
        SELECT `卡号`, `类型名称` 
        FROM 
            (
            SELECT 
                `卡号`, 
                `类型名称`, 
                ROW_NUMBER() OVER(PARTITION BY `卡号` ORDER BY `类型名称出现次数` DESC) row_number03 
            FROM 
                (
                SELECT 
                    `卡号`, 
                    `类型名称`, 
                    COUNT(*) `类型名称出现次数` 
                FROM SourceTable 
                GROUP BY `卡号`, `类型名称`
                ) d01 
            ) d02 
        WHERE row_number03 = 1
        ) d03
    ON agg.`卡号` = d03.`卡号`
    JOIN
        (
        SELECT `卡号`, `交易类型描述` 
        FROM 
            (
            SELECT 
                `卡号`, 
                `交易类型描述`, 
                ROW_NUMBER() OVER(PARTITION BY `卡号` ORDER BY `交易类型描述出现次数` DESC) row_number04 
            FROM 
                (
                SELECT 
                    `卡号`, 
                    `交易类型描述`, 
                    COUNT(*) `交易类型描述出现次数` 
                FROM SourceTable 
                GROUP BY `卡号`, `交易类型描述`
                ) e01 
            ) e02 
        WHERE row_number04 = 1
        ) e03
    ON agg.`卡号` = e03.`卡号`
    JOIN
        (
        SELECT `卡号`, `商品类型` 
        FROM 
            (
            SELECT 
                `卡号`, 
                `商品类型`, 
                ROW_NUMBER() OVER(PARTITION BY `卡号` ORDER BY `商品类型出现次数` DESC) row_number05 
            FROM 
                (
                SELECT 
                    `卡号`, 
                    `商品类型`, 
                    COUNT(*) `商品类型出现次数` 
                FROM SourceTable 
                GROUP BY `卡号`, `商品类型`
                ) f01 
            ) f02 
        WHERE row_number05 = 1
        ) f03
    ON agg.`卡号` = f03.`卡号`    
    JOIN
        (
        SELECT `卡号`, `商品种类名称` 
        FROM 
            (
            SELECT 
                `卡号`, 
                `商品种类名称`, 
                ROW_NUMBER() OVER(PARTITION BY `卡号` ORDER BY `商品种类名称出现次数` DESC) row_number06 
            FROM 
                (
                SELECT 
                    `卡号`, 
                    `商品种类名称`, 
                    COUNT(*) `商品种类名称出现次数` 
                FROM SourceTable 
                GROUP BY `卡号`, `商品种类名称`
                ) g01 
            ) g02 
        WHERE row_number06 = 1
        ) g03
    ON agg.`卡号` = g03.`卡号` 
    JOIN
        (
        SELECT `卡号`, `单位名称` 
        FROM 
            (
            SELECT 
                `卡号`, 
                `单位名称`, 
                ROW_NUMBER() OVER(PARTITION BY `卡号` ORDER BY `单位名称出现次数` DESC) row_number07 
            FROM 
                (
                SELECT 
                    `卡号`, 
                    `单位名称`, 
                    COUNT(*) `单位名称出现次数` 
                FROM SourceTable 
                GROUP BY `卡号`, `单位名称`
                ) h01 
            ) h02 
        WHERE row_number07 = 1
        ) h03
    ON agg.`卡号` = h03.`卡号` 
    ),
    
    
-- 行数  
RowNumber AS
    (
    SELECT ROUND(COUNT(1) * 0.9, 0)
    FROM 
    SourceTable
    ),
    

-- 阈值表
ReferenceTable AS
    (
    SELECT 
    'IC卡大额非油消费阈值' `指标名称`,
    `IC卡大额非油消费阈值` `阈值`
    FROM
        (
        SELECT `IC卡消费金额` `IC卡大额非油消费阈值`
        FROM 
            ( 
            SELECT 
                ABS(`实际金额`) `IC卡消费金额`,
                ROW_NUMBER() OVER(ORDER BY ABS(`实际金额`) ASC) row_number1
            FROM SourceTable
            WHERE 
                INT(`商品类型`) < 101000 
                AND `交易类型描述` LIKE '%IC%'
            ORDER BY `IC卡消费金额`, row_number1 ASC
            ) a11 
        WHERE row_number1 IN 
            (
            SELECT *
            FROM RowNumber
            ) 
        ) a1
    UNION
    SELECT 
    '个人卡汽油高额消费阈值',
    a2.`个人卡汽油高额消费阈值`
    FROM
        (
        SELECT `个人卡汽油消费金额` `个人卡汽油高额消费阈值`
        FROM 
            ( 
            SELECT 
                ABS(`实际金额`) `个人卡汽油消费金额`,
                ROW_NUMBER() OVER(ORDER BY ABS(`实际金额`) ASC) row_number2
            FROM SourceTable
            WHERE 
                `类型名称` LIKE '%个人卡%' 
                AND `商品种类名称` LIKE '%汽油%'
            ORDER BY `个人卡汽油消费金额`, row_number2 ASC
            ) a21 
        WHERE row_number2 IN 
            ( 
            SELECT * 
            FROM RowNumber 
            ) 
        ) a2 
    UNION
    SELECT
    '车队卡非油累计消费金额阈值',
    a3.`车队卡非油累计消费金额阈值`
        FROM
        (
        SELECT 
        `车队卡非油累计消费金额` `车队卡非油累计消费金额阈值`
        FROM 
            ( 
            SELECT 
            `卡号`, 
            ABS(SUM(`实际金额`)) `车队卡非油累计消费金额`,
            ROW_NUMBER() OVER(ORDER BY ABS(SUM(`实际金额`)) ASC) row_number3
            FROM SourceTable 
            WHERE 
                INT(`商品类型`) < 101000 
                AND `类型名称` LIKE '%车队%'
            GROUP BY `卡号`
            ORDER BY `车队卡非油累计消费金额`, row_number3 ASC
            ) a31 
        WHERE row_number3 IN 
            (
            SELECT *
            FROM RowNumber
            ) 
        ) a3 
    UNION
    SELECT
    '车队卡非油累计消费次数阈值',
    a4.`车队卡非油累计消费次数阈值`
    FROM
        (
        SELECT 
        `车队卡非油累计消费次数` `车队卡非油累计消费次数阈值`
        FROM 
            ( 
            SELECT
            `卡号`,
            COUNT(1) `车队卡非油累计消费次数`,
            ROW_NUMBER() OVER(ORDER BY COUNT(1) ASC) row_number4
            FROM SourceTable
            WHERE 
                INT(`商品类型`) < 101000 
                AND `类型名称` LIKE '%车队%'
            GROUP BY `卡号`
            ORDER BY `车队卡非油累计消费次数`, row_number4 ASC
            ) a41 
        WHERE row_number4 IN 
            (
            SELECT *
            FROM RowNumber
            ) 
        ) a4 
    UNION
    SELECT 
    '单日充值次数阈值',
    a5.`单日充值次数阈值`
    FROM
        (
        SELECT 
        `单日充值次数` `单日充值次数阈值`
        FROM 
            ( 
            SELECT
            `卡号`,
            `交易日期`,
            COUNT(1) `单日充值次数`,
            ROW_NUMBER() OVER(ORDER BY COUNT(1) ASC) row_number5
            FROM SourceTable
            WHERE `实际金额` > 0 
            GROUP BY 
                `卡号`,
                `交易日期`
            ORDER BY `单日充值次数`, row_number5 ASC
            ) a51 
        WHERE row_number5 IN 
            (
            SELECT *
            FROM RowNumber
            ) 
        ) a5 
    UNION
    SELECT 
    '单日消费次数阈值',
    a6.`单日消费次数阈值`
    FROM
        (
        SELECT 
        `单日消费次数` `单日消费次数阈值`
        FROM 
            ( 
            SELECT
            `卡号`,
            `交易日期`,
            COUNT(1) `单日消费次数`,
            ROW_NUMBER() OVER(ORDER BY COUNT(1) ASC) row_number6
            FROM SourceTable
            WHERE `实际金额` < 0 
            GROUP BY 
                `卡号`,
                `交易日期`
            ORDER BY `单日消费次数`, row_number6 ASC
            ) a61 
        WHERE row_number6 IN 
            (
            SELECT *
            FROM RowNumber
            ) 
        ) a6 
    UNION
    SELECT 
    '单日夜间消费次数阈值',
    a7.`单日夜间消费次数阈值`
    FROM
        (
        SELECT 
        `单日夜间消费次数` `单日夜间消费次数阈值`
        FROM 
            ( 
            SELECT 
            `卡号`,
            `交易日期`,
            COUNT(1) `单日夜间消费次数`,
            ROW_NUMBER() OVER(ORDER BY COUNT(1) ASC) row_number7
            FROM SourceTable
            WHERE 
                `实际金额` < 0 
                AND (SUBSTR(occuretime,12,2) >= 23 OR SUBSTR(occuretime,12,2) < 5)
            GROUP BY 
                `卡号`,
                `交易日期`
            ORDER BY `单日夜间消费次数`, row_number7 ASC
            ) a71 
        WHERE row_number7 IN 
            (
            SELECT *
            FROM RowNumber
            ) 
        ) a7 
    UNION
    SELECT 
    '车队卡柴油消费阈值',
    a8.`车队卡柴油消费阈值`
    FROM
        (
        SELECT 
        `车队卡柴油消费` `车队卡柴油消费阈值`
        FROM 
            ( 
            SELECT 
            ABS(`实际金额`) `车队卡柴油消费`,
            ROW_NUMBER() OVER(ORDER BY ABS(`实际金额`) ASC) row_number8
            FROM SourceTable
            WHERE 
                `商品种类名称` LIKE '%柴油%' 
                AND `类型名称` LIKE '%车队%'
            ORDER BY `车队卡柴油消费`, row_number8 ASC
            ) a81 
        WHERE row_number8 IN 
            (
            SELECT *
            FROM RowNumber
            ) 
        ) a8 
    ),

    
-- 聚合指标+业务规则指标
ParameterTable AS
    (
    SELECT 
        AggTable.*,
        p12.`IC卡大额非油消费次数`,
        p22.`个人卡汽油单笔高额消费次数`,
        p33.`个人卡消费过油品类型数量`,
        p43.`个人卡消费过汽油品类型数量`,
        p52.`车队卡非油累计消费金额是否过高`,
        p62.`车队卡非油累计消费次数是否过多`,
        p71.`车队卡累计消费次数`,
        p71.`车队卡累计消费金额`,
        p81.`个人卡累计充值金额`,
        p93.`单日充值次数过多次数`,
        p103.`车队卡汽油消费次数占比`,
        p103.`车队卡柴油消费次数占比`,
        p103.`车队卡天然气消费次数占比`,
        p113.`单日消费次数过多次数`,
        p121.`车队卡汽油累计消费次数`,
        p121.`车队卡汽油累计消费金额`,
        p133.`单日夜间消费次数过多次数`,
        p141.`个人卡夜间消费总次数`,
        p141.`个人卡夜间消费总金额`,
        p152.`个人卡消费过加油站站点数量`,
        p163.`非油累计消费金额`,
        p163.`非油消费占总消费占比`,
        p172.`车队卡柴油是否高额消费`
    FROM AggTable
    LEFT JOIN 
    (
    SELECT 
    `卡号`,
    SUM(`是否大额非油消费IC卡支付`) AS `IC卡大额非油消费次数` 
    FROM 
        ( 
        SELECT 
        `卡号`, 
        CASE WHEN ABS(`实际金额`) > `阈值` 
             THEN 1 
             ELSE 0 
             END 
             `是否大额非油消费IC卡支付` 
        FROM SourceTable, ReferenceTable 
        WHERE INT(`商品类型`) < 101000 
        AND `交易类型描述` LIKE '%IC%' 
        AND `指标名称` = 'IC卡大额非油消费阈值'
        ) p11 
    GROUP BY `卡号`
    ) p12 
    ON AggTable.`卡号` = p12.`卡号`
    LEFT JOIN
    (
    SELECT 
    `卡号`,
    SUM(`是否汽油单笔高额消费`) AS `个人卡汽油单笔高额消费次数` 
    FROM 
        ( 
        SELECT 
        `卡号`, 
        CASE WHEN ABS(`实际金额`) > `阈值`
             THEN 1 
             ELSE 0 
             END 
             `是否汽油单笔高额消费` 
        FROM SourceTable, ReferenceTable 
        WHERE `类型名称` LIKE '%个人卡%' 
        AND `商品种类名称` LIKE '%汽油%'
        AND `指标名称` = '个人卡汽油高额消费阈值'
        ) p21 
    GROUP BY `卡号`
    ) p22 
    ON AggTable.`卡号` = p22.`卡号`
    LEFT JOIN
    (
    SELECT `卡号`, COUNT(*) `个人卡消费过油品类型数量`FROM  
        (
        SELECT `卡号`, `油品种类` FROM
                ( 
                SELECT `卡号`,
                CASE WHEN `商品种类名称` LIKE '%汽油%' THEN '汽油' 
                     WHEN `商品种类名称` LIKE '%柴油%' THEN '柴油' 
                     ELSE '天然气'
                     END `油品种类` 
                FROM SourceTable 
                WHERE 
                    `类型名称` LIKE '%个人卡%' 
                    AND INT(`商品类型`) >= 101000
                ) p31 
        GROUP BY `卡号`, `油品种类` 
        ) p32 
    GROUP BY `卡号` 
    ) p33 
    ON AggTable.`卡号` = p33.`卡号`
    LEFT JOIN
    (
    SELECT `卡号`, COUNT(*) `个人卡消费过汽油品类型数量` 
    FROM  
        (
        SELECT `卡号`, `商品种类名称` FROM
                ( 
                SELECT `卡号`,
                `商品种类名称`
                FROM SourceTable 
                WHERE 
                    `类型名称` LIKE '%个人%' 
                    AND `商品种类名称` LIKE '%汽油%'
                ) p41 
        GROUP BY `卡号`, `商品种类名称` 
        ) p42
    GROUP BY `卡号`
    ) p43
    ON AggTable.`卡号` = p43.`卡号`
    LEFT JOIN
    (
    SELECT 
    `卡号`, 
    CASE WHEN ABS(`车队卡非油累计消费金额`) > `阈值` 
         THEN 1 
         ELSE 0 
         END `车队卡非油累计消费金额是否过高`
    FROM 
        (
        SELECT 
        `卡号`,
        SUM(`实际金额`) `车队卡非油累计消费金额`
        FROM SourceTable 
        WHERE 
            INT(`商品类型`) < 101000
            AND `类型名称` LIKE '%车队%'
        GROUP BY `卡号`
        ) p51, ReferenceTable
    WHERE `指标名称` = '车队卡非油累计消费金额阈值'
    ) p52 
    ON AggTable.`卡号` = p52.`卡号`
    LEFT JOIN
    (
    SELECT 
    `卡号`, 
    CASE WHEN `车队卡非油累计消费次数` > `阈值` 
         THEN 1 
         ELSE 0 
         END `车队卡非油累计消费次数是否过多`
    FROM 
        (
        SELECT 
        `卡号`,
        COUNT(1) `车队卡非油累计消费次数`
        FROM SourceTable 
        WHERE 
            INT(`商品类型`) < 101000
            AND `类型名称` LIKE '%车队%'
        GROUP BY `卡号`
        ) p61, ReferenceTable
    WHERE `指标名称` = '车队卡非油累计消费次数阈值'
    ) p62
    ON AggTable.`卡号` = p62.`卡号`
    LEFT JOIN
    (
    SELECT
    `卡号`, 
    ABS(SUM(`实际金额`)) `车队卡累计消费金额`, 
    COUNT(1) `车队卡累计消费次数`
    FROM SourceTable 
    WHERE 
        `实际金额` < 0 
        AND `类型名称` LIKE '%车队%' 
    GROUP BY `卡号`
    ) p71
    ON AggTable.`卡号` = p71.`卡号`
    LEFT JOIN
    (
    SELECT 
    `卡号`, 
    ABS(SUM(`实际金额`)) AS `个人卡累计充值金额`
    FROM SourceTable 
    WHERE 
        `实际金额` > 0 
        AND `类型名称` LIKE '%个人%' 
    GROUP BY `卡号`
    ) p81
    ON AggTable.`卡号` = p81.`卡号`
    LEFT JOIN
    (
    SELECT 
    `卡号`,
    SUM(`单日充值次数是否过多`) `单日充值次数过多次数`
    FROM
        ( 
        SELECT 
        `卡号`, 
        CASE WHEN `单日充值次数` > `阈值` 
             THEN 1 
             ELSE 0 
             END AS `单日充值次数是否过多` 
        FROM 
            ( 
            SELECT 
            `卡号`,
            `交易日期`,
            COUNT(*) AS `单日充值次数` 
            FROM SourceTable 
            WHERE 
                `实际金额` > 0 
            GROUP BY 
                `卡号`, 
                `交易日期` 
            ) p91, ReferenceTable
        WHERE `指标名称` = '单日充值次数阈值'
        ) p92 
    GROUP BY `卡号`
    ) p93
    ON AggTable.`卡号` = p93.`卡号`
    LEFT JOIN
    (
    SELECT 
        `卡号`, 
        `汽油消费总次数`/`总消费次数` AS `车队卡汽油消费次数占比`, 
        `柴油消费总次数`/`总消费次数` AS `车队卡柴油消费次数占比`, 
        `天然气消费总次数`/`总消费次数` AS `车队卡天然气消费次数占比` 
    FROM
        (
        SELECT 
            `卡号`,
            COUNT(1) `总消费次数`, 
            SUM(`是否汽油消费`) `汽油消费总次数`, 
            SUM(`是否柴油消费`) `柴油消费总次数`, 
            SUM(`是否天然气消费`) `天然气消费总次数` FROM 
            ( 
            SELECT
                *,
                CASE WHEN `商品种类名称` LIKE '%汽油%' THEN 1 ELSE 0 END `是否汽油消费`,
                CASE WHEN `商品种类名称` LIKE '%柴油%' THEN 1 ELSE 0 END `是否柴油消费`,
                CASE WHEN `商品种类名称` LIKE '%天然气%' THEN 1 ELSE 0 END `是否天然气消费`
            FROM SourceTable 
            WHERE 
                `类型名称` LIKE '%车队卡%' 
                AND INT(`商品类型`) >= 101000
            ) p101 
        GROUP BY `卡号` 
        ) p102 
    ) p103 
    ON AggTable.`卡号` = p103.`卡号`
    LEFT JOIN
    (
    SELECT 
    `卡号`,
    SUM(`单日消费次数是否过多`) `单日消费次数过多次数`
    FROM
        ( 
        SELECT 
        `卡号`, 
        CASE WHEN `单日消费次数` > `阈值` 
             THEN 1 
             ELSE 0 
             END AS `单日消费次数是否过多` 
        FROM 
            ( 
            SELECT 
            `卡号`,
            `交易日期`,
            COUNT(*) AS `单日消费次数` 
            FROM SourceTable 
            WHERE 
                `实际金额` < 0 
            GROUP BY 
                `卡号`, 
                `交易日期` 
            ) p111, ReferenceTable
        WHERE `指标名称` = '单日消费次数阈值'
        ) p112 
    GROUP BY `卡号`
    ) p113
    ON AggTable.`卡号` = p113.`卡号`
    LEFT JOIN
    (
    SELECT 
        `卡号`,
        COUNT(*) AS `车队卡汽油累计消费次数`,
        ABS(SUM(`实际金额`)) AS `车队卡汽油累计消费金额`
    FROM SourceTable 
    WHERE 
        `类型名称` LIKE '%车队卡%' 
        AND `商品种类名称` LIKE '%汽油%' 
    GROUP BY `卡号`
    ) p121
    ON AggTable.`卡号` = p121.`卡号`
    LEFT JOIN
    (
    SELECT 
        `卡号`,
        SUM(`是否单日夜间消费次数过多`) AS `单日夜间消费次数过多次数` FROM 
        (
        SELECT 
            `卡号`, 
            CASE WHEN `单日夜间消费次数` > `阈值` THEN 1 ELSE 0 END AS `是否单日夜间消费次数过多`
        FROM 
            (
            SELECT 
                 `卡号`, 
                 `交易日期`,
                 COUNT(*) AS `单日夜间消费次数` 
            FROM SourceTable 
            WHERE 
                `实际金额` < 0 
                AND (SUBSTR(occuretime,12,2) >= 23 OR SUBSTR(occuretime,12,2) < 5) 
            GROUP BY 
                `卡号`, 
                `交易日期` 
            ) p131, ReferenceTable
        WHERE `指标名称` = '单日夜间消费次数阈值'
        ) p132
    GROUP BY `卡号`
    ) p133
    ON AggTable.`卡号` = p133.`卡号`
    LEFT JOIN
    (
    SELECT 
        `卡号`,
        COUNT(*) `个人卡夜间消费总次数`,
        ABS(SUM(`实际金额`)) `个人卡夜间消费总金额` 
    FROM SourceTable 
    WHERE 
        `类型名称` LIKE '%个人卡%' 
        AND (HOUR(occuretime) > 23 OR HOUR(occuretime) < 5) 
    GROUP BY `卡号`
    ) p141
    ON AggTable.`卡号` = p141.`卡号`
    LEFT JOIN
    (
    SELECT 
        `卡号`,
        COUNT(`站点名称`) AS `个人卡消费过加油站站点数量`
        FROM 
        (
        SELECT 
            `卡号`,
            `站点名称` 
        FROM SourceTable 
        WHERE `类型名称` LIKE '%个人%'
        GROUP BY  
            `卡号`,
            `站点名称` 
        ) p151 
    GROUP BY `卡号`
    ) p152
    ON AggTable.`卡号` = p152.`卡号`
    LEFT JOIN
    (
    SELECT 
        p161.`卡号`,
        `非油累计消费金额` `非油累计消费金额`,
        `非油累计消费金额`/`总累计消费金额` `非油消费占总消费占比` 
    FROM
        (
        SELECT 
            `卡号`,
            ABS(SUM(`实际金额`)) `非油累计消费金额` 
        FROM SourceTable 
        WHERE 
            INT(`商品类型`) < 101000 
            AND `实际金额` < 0
        GROUP BY `卡号`
        ) p161 
    JOIN
        (
        SELECT 
            `卡号`, 
            ABS(SUM(`实际金额`)) `总累计消费金额` 
        FROM SourceTable
        WHERE `实际金额` < 0 
        GROUP BY `卡号`
        ) p162  
    ON p161.`卡号` = p162.`卡号`
    ) p163
    ON AggTable.`卡号` = p163.`卡号`
    LEFT JOIN
    (
    SELECT 
        `卡号`, 
        CASE WHEN ABS(`车队卡柴油累计消费金额`) > `阈值` THEN 1 ELSE 0 END `车队卡柴油是否高额消费` 
    FROM 
        (
        SELECT 
            `卡号`, 
            SUM(`实际金额`) `车队卡柴油累计消费金额` 
        FROM app_gsms_kjyxxb_2020_all 
        WHERE 
            `商品种类名称` LIKE '%柴油%'  
            AND `类型名称` LIKE '%车队%' 
        GROUP BY `卡号`
        ) p171, ReferenceTable
    WHERE `指标名称` = '车队卡柴油消费阈值'
    ) p172
    ON AggTable.`卡号` = p172.`卡号`
)
    
SELECT * FROM ParameterTable